In [1]:
from pickle import GLOBAL
import pinocchio as pin
import numpy as np
import time
import math
import sys
from numpy.linalg import norm, inv, pinv, det, matrix_rank
import pandas as pd

In [2]:
pinocchio_model_dir = '/home/unknown/Documents/Posdoc/ModelsCads/urdf' 
mesh_dir = pinocchio_model_dir + '/kinova_description/meshes/'
urdf_model_path =pinocchio_model_dir + '/j2s6s200_simple.urdf'
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir)

In [3]:
frameIndex = model.getFrameId('j2s6s200_end_effector')
jointIndex = model.frames[frameIndex].parent
# Create data required by the algorithms
data     = model.createData()

In [12]:
def run_robot(): 
    # for runing the simulation
    t = 0
    dt = 1e-1
    
    # for graphs
    E1_list = []
    manu_list = []
    q_list = [] #q
    pose_list = []
    
    a = np.arange(47,180,1)
    b = np.arange(19,180,1)
    c = np.arange(65,295,1)

    ## for runing the simulation
    for k  in np.nditer(a):
        t += dt
        tic = time.time()
        q2  = np.deg2rad(k)
        for n in np.nditer(b):
            q3  = np.deg2rad(n)
            for p in np.nditer(c):
                q5  = np.deg2rad(p)
                q =np.array([0, q2, q3, np.pi, q5,0]) # Natural position as in the real case
                pin.forwardKinematics(model,data,q)
                pin.updateFramePlacements(model,data)
                OM_act = data.oMf[frameIndex] # This is the pose of the end-effector refered in the frame of the base

                pin.computeJointJacobians(model, data, q)
                J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED) # in the base frame
                Jbl = J[:3] # linear part of the Jacobian
                #Jbl = check_matrix(Jbl)
                Al = Jbl@Jbl.T
                Al_det = (det(Al))

                if Al_det < 1e-15:  
                   Al_det = 0.0
                
                if Al_det == 0.0:
                    E1 = math.nan
                else:
                    Al_inv = inv(Al)
                    E1 = norm(Al_inv,ord=2)
                manip_linear = np.sqrt(Al_det)
                pos = np.array([OM_act.translation[0],OM_act.translation[1],OM_act.translation[2]])

                q_list.append(q)
                pose_list.append(pos)
                manu_list.append(manip_linear)
                E1_list.append(E1)


                toc = time.time()
                ellapsed = toc - tic
                dt_sleep = max(0,dt - (ellapsed))
                time.sleep(dt_sleep)

    manu_list = np.array(manu_list)
    E1_list = np.array(E1_list)
    q_list = np.array(q_list)
    pose_list = np.array(pose_list)
    
    return  pose_list, q_list, manu_list, E1_list

In [56]:
def check_matrix(M, eps=1e-15):
    ncol = M.shape[1]
    NC = 6
    for i in range(math.floor((ncol - 1) / NC + 1)):
            cmin = i * 6
            cmax = (i + 1) * 6
            cmax = ncol if ncol < cmax else cmax
            #print("Columns %s through %s" % (cmin, cmax - 1))
            #print()
            for r in range(M.shape[0]):
                sys.stdout.write("  ")
                for c in range(cmin, cmax):
                    if abs(M[r,c])>eps: 
                        M[r,c] = M[r,c]
                    else: 
                        M[r,c] =0
                #print()
            #print()
    return M

In [8]:
### Only for tests ####
q =np.array([np.pi, np.pi, np.pi, np.pi, np.pi,0])
#q =np.array([0, 2.9, 1.3, -2.07, 1.4,0]) # Natural position as in the real case
pin.forwardKinematics(model,data,q)
pin.updateFramePlacements(model,data)
OM_act = data.oMf[frameIndex] # This is the pose of the end-effector refered in the frame of the base

pin.computeJointJacobians(model, data, q)
J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED) # in the base frame
Jbl = J[:3] # linear part of the Jacobian
#Jbl = check_matrix(Jbl)
Al = Jbl@Jbl.T
Al_det = (det(Al)) 
if Al_det < 1e-15:  
    Al_det = 0.0
if Al_det == 0.0:
   Al_inv = np.array([(math.nan, math.nan,math.nan),
                      (math.nan, math.nan,math.nan),
                      (math.nan, math.nan,math.nan)])
   E1 = math.nan
else:
   Al_inv = inv(Al)
   E1 = norm(Al_inv,ord=2)

manip_linear = np.sqrt(Al_det)

print(E1)
#print(Al_inv)



nan


In [13]:
pose, q, manip, E1 = run_robot()

In [14]:
pd.DataFrame(pose).to_csv('pose_workspaceXZE1.csv', index_label = "Index", header  = ['x','y','z'])    

In [15]:
pd.DataFrame(manip).to_csv('manip_workspaceXZE1.csv', index_label = "Index", header  = ['manip'])    

In [16]:
pd.DataFrame(q).to_csv('qs_workspaceXZE1.csv', index_label = "Index", header  = ['q1','q2','q3','q4','q5','q6'])    